<a href="https://colab.research.google.com/github/Sriyati5/Biodata/blob/main/Salinan_dari_Model_AI_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===================================================================
# LANGKAH 0: MEMBUAT FILE CSV SECARA OTOMATIS
# ===================================================================
# Data yang akan kita tulis ke file CSV
csv_data = """teks,label
"Kameranya mantap, sangat jernih dan tajam. Baterai juga awet seharian. Puas banget!","positif"
"Performa gamingnya luar biasa, settingan rata kanan lancar jaya. Sangat direkomendasikan.","positif"
"Saya agak kecewa dengan daya tahan baterainya, cepat sekali habis padahal pemakaian normal.","negatif"
"Desainnya bagus dan premium, tapi layarnya kurang cerah kalau di bawah matahari.","netral"
"Ini ponsel terbaik di kelas harganya. Fitur lengkap, tidak ada keluhan berarti.","positif"
"Paket datang dengan selamat, packing rapi. Ponselnya berfungsi dengan baik.","netral"
"Sering banget nge-freeze setelah update software terakhir. Sangat mengganggu pekerjaan saya.","negatif"
"Kenapa tidak ada charger di dalam kotak? Benar-benar keputusan yang buruk dari brand ini.","negatif"
"Speaker stereonya kencang dan jernih, asyik buat nonton film.","positif"
"Bodi belakangnya gampang sekali tergores, padahal baru seminggu dipakai.","negatif"
"""

# Menulis data ke file bernama 'ulasan_produk.csv'
with open('ulasan_produk.csv', 'w') as f:
    f.write(csv_data)

print("File 'ulasan_produk.csv' berhasil dibuat secara otomatis.\n")

File 'ulasan_produk.csv' berhasil dibuat secara otomatis.



In [3]:
# ===================================================================
# LANGKAH 1: INSTALASI PUSTAKA YANG DIBUTUHKAN
# ===================================================================
# Opsi -q digunakan agar output instalasi tidak terlalu ramai (quiet)


!pip install -q transformers datasets scikit-learn torch accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
# ===================================================================
# LANGKAH 2: IMPORT DAN PROSES DATA
# ===================================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import numpy as np
import evaluate

print("Memuat data dari ulasan_produk.csv...")
# Mengubah pemisah dari ';' menjadi ','
df = pd.read_csv('ulasan_produk.csv', sep=',')

# Hapus baris dengan nilai yang hilang di kolom 'label'
df.dropna(subset=['label'], inplace=True)

# Membuat label dalam bentuk angka
labels = df['label'].unique().tolist()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}
df['label_id'] = df['label'].map(label2id)

# Membagi data menjadi data latih dan uji
# Mengubah test_size menjadi 0.3 untuk memastikan setidaknya 3 sampel di test set
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])

# Mengubah pandas DataFrame menjadi Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
raw_datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(f"Data latih: {len(train_dataset)} | Data uji: {len(test_dataset)}\n")

# ===================================================================
# LANGKAH 3: MEMUAT MODEL DASAR DAN TOKENIZER
# ===================================================================
print("Memuat tokenizer dan model dasar IndoBERT...")
model_checkpoint = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    output_attentions=False, # Tambahkan ini
    output_hidden_states=False # Tambahkan ini
)
print("Tokenisasi dataset...")
def tokenize_function(examples):
    # Ensure that the 'teks' column exists in the examples
    if "teks" not in examples:
        raise ValueError("Column 'teks' not found in the dataset.")
    return tokenizer(examples["teks"], padding="max_length", truncation=True)

# Pass remove_columns argument to the map function to remove original columns
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=["teks", "label", "__index_level_0__"])

# Ganti nama kolom 'label_id' menjadi 'labels' agar sesuai dengan yang diharapkan Trainer
tokenized_datasets = tokenized_datasets.rename_column("label_id", "labels")

print("Model dan tokenizer siap.\n")


# ===================================================================
# LANGKAH 4: FINE-TUNING MODEL
# ===================================================================
print("Memulai proses Fine-Tuning...")

# Fungsi untuk menghitung metrik evaluasi
metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

# Menentukan argumen training
training_args = TrainingArguments(
    output_dir="./hasil_fine_tuning",
    eval_strategy="epoch", # Mengganti evaluation_strategy menjadi eval_strategy
    num_train_epochs=5, # Kita tambah sedikit epoch untuk hasil lebih baik
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    logging_steps=1,
    save_strategy="epoch", # Simpan model di setiap epoch
    load_best_model_at_end=True, # Muat model terbaik di akhir training
)

# Membuat objek Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Mulai training! 🧠
trainer.train()
print("Fine-tuning selesai.\n")

# ===================================================================
# LANGKAH 5: EVALUASI DAN PENGGUNAAN MODEL
# ===================================================================
print("Mengevaluasi dan menggunakan model yang sudah di-fine-tune...")
eval_results = trainer.evaluate()
print(f"\nHasil Evaluasi Final pada Test Set: \n{eval_results}\n")

# Menyimpan model terbaik
trainer.save_model("./model_sentimen_terbaik")
print("Model terbaik telah disimpan di folder 'model_sentimen_terbaik'.\n")

# Contoh penggunaan model final untuk prediksi
print("===== CONTOH PREDIKSI DENGAN MODEL BARU =====")
custom_predictor = pipeline("sentiment-analysis", model="./model_sentimen_terbaik")
contoh_teks_baru = [
    "HP ini keren banget, tidak nyesel beli!",
    "Baterainya parah, cepat habis.",
    "Biasa saja, tidak ada yang istimewa."
]
for teks in contoh_teks_baru:
    hasil = custom_predictor(teks)
    print(f"Teks: '{teks}' -> Sentimen: {hasil[0]['label']} (Skor: {hasil[0]['score']:.2f})")

Memuat data dari ulasan_produk.csv...
Data latih: 7 | Data uji: 3

Memuat tokenizer dan model dasar IndoBERT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Model dan tokenizer siap.

Memulai proses Fine-Tuning...


/tmp/ipython-input-4-1656879474.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: